In [81]:
# !pip install img2vec_pytorch
# conda install faiss-cpu -c pytorch -n torch

In [2]:
from pathlib import Path
from img2vec_pytorch import Img2Vec
from PIL import Image
import faiss
import numpy as np
import os
import pickle

In [3]:
image_path = '../../data'
artifact_path = 'artifacts'

In [4]:
img_paths = []
for path in Path(image_path).rglob('*.jpg'):
    img_paths.append(path)
# img_paths[:5]

In [5]:
img2vec = Img2Vec(cuda=False)
vectors = img2vec.get_vec([Image.open(ipath) for ipath in img_paths])

C:\Users\sparsh.agarwal\.conda\envs\torch\lib\site-packages\torchvision\transforms\transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


(10, 512)


In [6]:
item_vectors = {ipath.stem:vectors[iindex] for iindex, ipath in enumerate(img_paths)}

In [7]:
item_paths = {ipath.stem:ipath for ipath in img_paths}

In [8]:
pickle.dump(item_vectors, open(os.path.join(artifact_path,"item_vectors.p"), "wb"))

In [10]:
d=512

In [11]:
index = faiss.IndexFlatL2(d)
index.add(np.array(list(item_vectors.values())))
print(index.ntotal)

10


In [13]:
# print(index.is_trained)
k = 4 # we want to see 4 nearest neighbors
D, I = index.search(vectors[:5], k) # sanity check
print(I)
print(D)

[[0 2 4 1]
 [1 2 0 4]
 [2 1 0 4]
 [3 4 5 6]
 [4 3 2 5]]
[[  0.      327.47327 401.84265 411.65393]
 [  0.      206.0236  411.65393 428.9434 ]
 [  0.      206.0236  327.47327 352.76587]
 [  0.      273.33212 343.14578 362.2754 ]
 [  0.      273.33212 352.76587 392.9403 ]]


https://github.com/facebookresearch/faiss/wiki/Getting-started

In [75]:
def topk_similar(itemid, topk=2):
    _, I = index.search(item_vectors[str(itemid)].reshape(1,-1), topk+1)
    return [list(item_vectors.keys())[i] for i in I.flatten()][1:]

In [80]:
topk_similar('10006')

['10008', '10005']

In [23]:
def topk_distance(itemid, topk=2):
    D, I = index.search(item_vectors[str(itemid)].reshape(1,-1), topk+1)
    return {list(item_vectors.keys())[i]:j for (i,j) in zip(I.flatten(),D.flatten())}

In [25]:
topk_distance('10006')

{'10006': 0.0, '10008': 247.23978, '10005': 247.81348}

In [24]:
# D, I = index.search(item_vectors[str('10001')].reshape(1,-1), 2+1)
# [list(item_vectors.keys())[i] for i in I.flatten()][1:]
# {i:j for (i,j) in zip(I.flatten(),D.flatten())}

In [90]:
!python itemrep_training.py

(10, 512)


C:\Users\sparsh.agarwal\.conda\envs\torch\lib\site-packages\torchvision\transforms\transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [95]:
import logging
# log = logging.getLogger("my")
# log.setLevel(logging.INFO)
logging.info("Hello, world")

INFO:root:Hello, world


In [106]:
from itemrep_inference import topk_similar as topsim

INFO:root:Loading item vectors
INFO:root:Loading indexer
